In [1]:
!pwd

/home/stang/new_data_sources


In [2]:
import sys
sys.path.insert(0, 'stats-core/')
from vehicle import *
from vehicle_stats import *
from datetime import datetime, date
import subprocess
import findspark

# spark location on namenode server
findspark.init("/usr/hdp/current/spark2-client")
import pyspark
from pyspark.sql import HiveContext
from pyspark.sql.functions import col, when

import zipfile
zf = zipfile.ZipFile('veh.zip', 'w')
for f in ['vehicle.py', 'vehicle_stats.py', 'vehicle_utils.py']:
    zf.write('stats-core/' + f, f)
zf.close()

In [3]:
# configs
conf = pyspark.SparkConf().setAll([('spark.app.name', 'guobiao_daily_stats_run'),
                                   ('spark.master', 'yarn'),
                                   ('spark.submit.deployMode', 'client'),
                                   ('spark.executor.memory', '10g'),
                                   ('spark.memory.fraction', '0.7'),
                                   ('spark.executor.cores', '3'),
                                   ('spark.executor.instances', '20'),
                                   ('spark.yarn.am.memory', '10g')])
conf1 = pyspark.SparkConf().setAll([('spark.app.name', 'guobiao_export_to_hive'),
                                    ('spark.master', 'local'),
                                    ('spark.executor.memory', '10g'),
                                    ('spark.memory.fraction', '0.7'),
                                    ('spark.executor.cores', '3')])

In [4]:
pd.options.display.max_columns = 100

In [5]:
df = pd.DataFrame()
for i in range(4):
    veh = Vehicle('/gaei/gacrnd/data/guobiao/csv/d=20180101/part-r-0000{}'.format(i))
    df = pd.concat([df, veh.df])
    print(df.shape)

(16452, 79)
(32904, 79)
(49353, 79)
(65811, 79)


In [6]:
df_orig = df

In [7]:
df_orig.shape

(65811, 79)

In [9]:
df_orig.tail()

,vin,vintype,ts,veh_st,veh_charge_st,veh_runmode,veh_spd,veh_odo,veh_volt,veh_curr,veh_soc,veh_dcdc_st,veh_gear,veh_insulation,veh_acce_pedal,veh_brake_pedal,dm_seq,dm_st,dm_ctl_temp,dm_spd,dm_torq,dm_temp,dm_ctl_volt,dm_ctl_dc_curr,eng_st,eng_spd,eng_consumption,loc_st,loc_lon,loc_lat,data_batt_subsys_volt_highest_seq,data_batt_sc_volt_highest_seq,data_batt_sc_volt_highest,data_batt_subsys_volt_lowest_seq,data_batt_sc_volt_lowest_seq,data_batt_sc_volt_lowest,data_batt_subsys_temp_highest_seq,data_batt_temp_probe_highest_seq,data_batt_temp_highest,data_batt_subsys_temp_lowest_seq,data_batt_temp_probe_lowest_seq,data_batt_temp_lowest,alm_lvl_higest,alm_common_flag,alm_common_temp_diff,alm_common_temp_high,alm_common_esd_high,alm_common_esd_low,alm_common_soc_low,alm_common_sc_high,alm_common_sc_low,alm_common_soc_high,alm_common_soc_hop,alm_common_esd_unmatch,alm_common_sc_consistency,alm_common_insulation,alm_common_dcdc_temp,alm_common_brk,alm_common_dcdc_st,alm_common_dmc_temp,alm_common_hvil_st,alm_common_dm_temp,alm_common_esd_charge_over,alm_esd_cnt,alm_esd_list,alm_dm_cnt,alm_dm_list,alm_eng_cnt,alm_eng_list,alm_others_cnt,alm_others_list,esd_volt_subsys_seq,esd_volt,esd_curr,esd_sc_cnt,esd_frame_start,esd_frame_sc_cnt,esd_sc_volt_list,esd_sc_temp_list
16453,LMGFJ1S58H1000205,A51,2018-01-01 23:59:15+08:00,1,3,2,97.2,47310.0,320.0,-1.5,21,1,101110,1000,NaN,NaN,1.0,1.0,14.0,6307.0,0.0,62.0,321.0,-2.0,1.0,2708.0,0.0,0,120.839763,29.745911,1,60,4.657,1,11,4.646,1,4,23,1,12,21,0,0000000000000000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0,NaN,0,NaN,NaN,320.9,1.9,88,1,88,"[3.654, 3.654, 3.654, 3.654, 3.654, 3.655, 3.6...","[22, 23, 23, 22, 23, 22, 21, 21, 21, 22, 22, 2..."
16454,LMGFJ1S59H1000052,A51,2018-01-01 23:59:25+08:00,1,3,1,0.0,15282.0,316.0,5.0,21,1,011110,1000,NaN,NaN,1.0,1.0,24.0,2.0,11.0,64.0,315.0,2.0,2.0,0.0,0.0,0,120.124093,30.296846,1,67,4.606,1,46,4.598,1,3,26,1,12,23,0,0000000000000000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0,NaN,0,NaN,NaN,316.5,5.3,88,1,88,"[3.6010000000000004, 3.6010000000000004, 3.6, ...","[23, 25, 26, 24, 25, 24, 23, 23, 23, 24, 25, 2..."
16455,LMGFJ1S54H1000489,A51,2018-01-01 23:59:35+08:00,2,1,1,0.0,3625.0,363.0,-6.5,93,1,001111,1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,113.444877,22.952329,1,75,5.140,1,84,5.131,1,1,28,1,7,26,0,0000000000000000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0,NaN,0,NaN,NaN,363.0,6.7,88,1,88,"[4.137, 4.137, 4.137, 4.135, 4.135, 4.137, 4.1...","[28, 27, 27, 27, 27, 27, 26, 26, 26, 27, 27, 2..."
16456,LMGHP1S50H1001100,A5HEV,2018-01-01 23:59:45+08:00,2,1,1,0.0,1293.0,348.0,-6.5,70,1,001111,1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,116.160042,39.835327,1,52,4.878,1,27,4.868,1,10,5,1,1,2,0,0000000000000000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0,NaN,0,NaN,NaN,348.4,6.8,90,1,90,"[3.872, 3.873, 3.8739999999999997, 3.873999999...","[2, 2, 2, 3, 3, 3, 3, 4, 4, 5, 5, 4, 4, 4, 4, ..."
16457,LMGFJ1S52H1000104,A51,2018-01-01 23:59:55+08:00,1,3,1,3.9,38291.0,316.0,1.5,17,1,011110,1000,NaN,NaN,1.0,1.0,38.0,241.0,0.0,56.0,317.0,0.0,2.0,0.0,0.0,0,121.423644,31.155651,1,57,4.615,1,88,4.589,1,11,23,1,7,21,0,0000000000000000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0,NaN,0,NaN,NaN,316.5,2.6,88,1,88,"[3.6039999999999996, 3.6060000000000003, 3.606...","[22, 23, 23, 22, 22, 22, 22, 21, 21, 23, 23, 2..."


In [37]:
df_orig.columns

Index([u'vin', u'vintype', u'ts', u'esd_sc_temp_list'], dtype='object')

_______

In [111]:
d = 20180213
day = pd.to_datetime(d, format='%Y%m%d').date()
day = '{}-{}-{}'.format(day.year, day.month, day.day)
day

'2018-2-13'

In [112]:
df = pd.read_csv('guobiao_stats/{}/stats_{}_daily.csv'.format(d, d))

In [113]:
df[df['time_label'] != day]

,SOC_count,SOC_max,SOC_mean,SOC_min,SOC_std,acce_pedal_count,acce_pedal_max,acce_pedal_mean,acce_pedal_min,acce_pedal_std,battery_capacity,battery_temp_avg,battery_temp_max,battery_temp_min,battery_voltage_count,battery_voltage_max,battery_voltage_mean,battery_voltage_min,battery_voltage_std,brake_pedal_count,brake_pedal_max,brake_pedal_mean,brake_pedal_min,brake_pedal_std,cell_volt_diff_count,cell_volt_diff_max,cell_volt_diff_mean,cell_volt_diff_min,cell_volt_diff_std,charged_energy,charged_times,charging_hours,cum_elec_energy_consumption,cum_fuel_consumption,distance_driven,distance_on_battery,distance_on_fuel,driving_hours,elec_energy_efficiency,energy_throughput,fuel_efficiency_count,fuel_efficiency_max,fuel_efficiency_mean,fuel_efficiency_min,fuel_efficiency_std,odometer_offset,soc_range_charge_end_count,soc_range_charge_end_max,soc_range_charge_end_mean,soc_range_charge_end_min,soc_range_charge_end_std,soc_range_charge_start_count,soc_range_charge_start_max,soc_range_charge_start_mean,soc_range_charge_start_min,soc_range_charge_start_std,speed_average,speed_count,speed_diff_count,speed_diff_max,speed_diff_mean,speed_diff_min,speed_diff_std,speed_max,time_label,vin


In [35]:
df.shape

(2032, 3)

______

In [21]:
df_orig = df_orig[['vin', 'vintype', 'ts', 'esd_sc_temp_list']].copy()

In [22]:
df_orig.head()

,vin,vintype,ts,esd_sc_temp_list
0,LMGHP1S55H1000220,A5HEV,2017-12-06 04:12:55+08:00,"[22, 22, 22, 21, 21, 21, 21, 21, 21, 21, 21, 2..."
1,LMGHP1S55H1000220,A5HEV,2017-12-06 04:13:25+08:00,"[22, 22, 22, 21, 21, 21, 21, 21, 21, 21, 21, 2..."
2,LMGHP1S55H1000220,A5HEV,2017-12-06 04:15:15+08:00,"[22, 22, 22, 21, 21, 21, 21, 21, 21, 21, 21, 2..."
3,LMGHP1S55H1000220,A5HEV,2017-12-06 04:15:55+08:00,"[22, 22, 22, 21, 21, 21, 21, 21, 21, 21, 21, 2..."
4,LMGHP1S55H1000220,A5HEV,2017-12-06 04:16:35+08:00,"[22, 22, 22, 21, 21, 21, 21, 21, 21, 21, 21, 2..."


In [25]:
df_orig[df_orig['vin'] == 'LMGAJ1S28H1000883'].sort_values(by='ts')

,vin,vintype,ts,esd_sc_temp_list
2384,LMGAJ1S28H1000883,A2APHEV,2017-12-28 23:36:45+08:00,"[16, 16, 16, 15, 16, 16, 16, 15, 15, 15, 15, 1..."
11664,LMGAJ1S28H1000883,A2APHEV,2018-01-01 10:42:05+08:00,"[16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 15, 1..."
11692,LMGAJ1S28H1000883,A2APHEV,2018-01-01 10:46:45+08:00,"[16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 1..."
11715,LMGAJ1S28H1000883,A2APHEV,2018-01-01 10:51:05+08:00,"[16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 1..."
11719,LMGAJ1S28H1000883,A2APHEV,2018-01-01 10:51:15+08:00,"[16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 1..."
11947,LMGAJ1S28H1000883,A2APHEV,2018-01-01 11:28:15+08:00,"[18, 18, 17, 19, 17, 17, 18, 18, 17, 17, 17, 1..."


In [28]:
vin_filter = df_orig['vin'] == 'LMGAJ1S28H1000883'
date_filter = df_orig['ts'].dt.date == date(2017,12,29)
df_tmp = df_orig[(vin_filter) & (date_filter)].sort_values(by='ts')

In [29]:
np.nanmean(df_tmp['esd_sc_temp_list'].apply(np.nanmean))

/usr/lib/python2.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: Mean of empty slice
  """Entry point for launching an IPython kernel.


True

In [6]:
df.drop(labels='day', axis=1, inplace=True)

In [18]:
! ls /gaei/gacrnd/data/guobiao/csv/d=20180101/

part-r-00000  part-r-00001  part-r-00002  part-r-00003	_SUCCESS


In [7]:
df_orig = pd.read_csv('/gaei/gacrnd/data/guobiao/csv/d=20180101/part-r-00000', sep=',', header=None, dtype='str')

In [8]:
df_temp = df_orig.iloc[:, -1]

In [9]:
df_temp.head()

0     16 | 16 | 16 | 16 | 17 | 16 | 16 | 16 | 16 | ...
1     16 | 16 | 17 | 16 | 17 | 16 | 16 | 16 | 16 | ...
2     16 | 16 | 17 | 16 | 17 | 16 | 16 | 16 | 16 | ...
3     16 | 16 | 17 | 16 | 17 | 17 | 16 | 16 | 16 | ...
4     16 | 17 | 17 | 16 | 17 | 17 | 16 | 17 | 16 | ...
Name: 78, dtype: object

In [10]:
df_temp.shape

(400460,)

In [11]:
def list_to_numeric(x):
    """
    Converts a string with a list of integers separated by | to a numpy array:
    e.g.
    list_to_numeric('1|2|3')
    returns: np.array([1,2,3])
    """
    try:
        res = pd.to_numeric(x.split('|'))
    except:
        res = np.nan
    return res

In [14]:
df_temp = df_temp.apply(list_to_numeric)

In [15]:
df_temp.head()

0    [16, 16, 16, 16, 17, 16, 16, 16, 16, 16, 16, 1...
1    [16, 16, 17, 16, 17, 16, 16, 16, 16, 16, 16, 1...
2    [16, 16, 17, 16, 17, 16, 16, 16, 16, 16, 16, 1...
3    [16, 16, 17, 16, 17, 17, 16, 16, 16, 16, 16, 1...
4    [16, 17, 17, 16, 17, 17, 16, 17, 16, 16, 16, 1...
Name: 78, dtype: object

In [19]:
a = np.nanmean(df_temp.apply(np.nanmean))

In [20]:
type(a)

numpy.float64

In [17]:
df['battery_temp_avg'].head()

0    24.685242963356345
1      5.23047263681592
2     7.733333333333333
3    27.246593203230148
4    13.617313915857602
Name: battery_temp_avg, dtype: object

In [24]:
df['battery_temp_avg'].head()

0    24.685242963356345
1      5.23047263681592
2     7.733333333333333
3    27.246593203230148
4    13.617313915857602
Name: battery_temp_avg, dtype: object

In [28]:
df = df['battery_temp_avg']

In [31]:
df = pd.to_numeric(df, errors='coerce')

In [35]:
df.head()

0    24.685243
1     5.230473
2     7.733333
3    27.246593
4    13.617314
Name: battery_temp_avg, dtype: float64

In [26]:
df['battery_temp_avg'] = pd.to_numeric(df['battery_temp_avg'], errors='ignore')

In [27]:
df.head()

,SOC_count,SOC_max,SOC_mean,SOC_min,SOC_std,acce_pedal_count,acce_pedal_max,acce_pedal_mean,acce_pedal_min,acce_pedal_std,...,soc_range_charge_start_std,speed_average,speed_count,speed_diff_count,speed_diff_max,speed_diff_mean,speed_diff_min,speed_diff_std,speed_max,vin
0,1883.0,2.0,1.003717,1.0,0.060874,0.0,NaN,NaN,NaN,NaN,...,0.0,42.897337,1164.0,1882.0,105.2,-8.501594e-03,-80.9,11.532129,128.1,LMGAJ1S28H1000740
1,212.0,2.0,1.009434,1.0,0.096898,0.0,NaN,NaN,NaN,NaN,...,0.0,10.614692,211.0,211.0,29.6,3.157032e-17,-26.6,7.126911,63.0,LMGHP1S51H1000828
2,3.0,2.0,1.333333,1.0,0.577350,0.0,NaN,NaN,NaN,NaN,...,NaN,17.833333,3.0,2.0,-11.3,-2.110000e+01,-30.9,13.859293,42.2,LMGHP1S56H1001781
3,1486.0,2.0,1.000673,1.0,0.025941,0.0,NaN,NaN,NaN,NaN,...,0.0,49.678304,1415.0,1485.0,71.4,-1.435440e-17,-62.2,10.737531,122.9,LMGAJ1S25H1000503
4,206.0,2.0,1.014563,1.0,0.120088,0.0,NaN,NaN,NaN,NaN,...,0.0,19.062914,151.0,205.0,64.8,-9.804878e-02,-56.2,14.406970,64.8,LMGAJ1S8XH1001716


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2032 entries, 0 to 2031
Data columns (total 65 columns):
SOC_count                       2032 non-null float64
SOC_max                         1947 non-null float64
SOC_mean                        1947 non-null float64
SOC_min                         1947 non-null float64
SOC_std                         1928 non-null float64
acce_pedal_count                2032 non-null float64
acce_pedal_max                  0 non-null float64
acce_pedal_mean                 0 non-null float64
acce_pedal_min                  0 non-null float64
acce_pedal_std                  0 non-null float64
battery_capacity                1776 non-null float64
battery_temp_avg                1987 non-null object
battery_temp_max                1947 non-null float64
battery_temp_min                1947 non-null float64
battery_voltage_count           2032 non-null float64
battery_voltage_max             1947 non-null float64
battery_voltage_mean            1947 non-n

In [11]:
spark_df = hc.createDataFrame(df)

TypeError: Can not merge type <class 'pyspark.sql.types.StringType'> and <class 'pyspark.sql.types.DoubleType'>

In [ ]:
    print('EXPORTING TO HIVE...')
    for i, d in enumerate(dates):
        # check if stats has been calculated
        stats_file = os.path.join(dest_dir, str(d), 'stats_{}_{}.csv'.format(d, freq))
        if not os.path.exists(stats_file):
            print('{} does not exist, skipping'.format(stats_file))
            continue

        # check if file has been exported to Hive already
        out_flag_file = stats_file + '.done'
        if not force_run and os.path.exists(out_flag_file):
            print('{} has been exported to Hive before, skipping..'.format(stats_file))
            continue

        # take a break every 100 files, so that others can also run spark jobs
        if i > 0 and i % 100 == 1:
            sc.stop()
            sc = pyspark.SparkContext(conf=conf1)
            hc = HiveContext(sc)

        # start exporting stats (csv file) to Hive
        ts = pd.to_datetime(datetime.now())
        print('{}: exporting {} to Hive table guobiao_tsp_tbls.{}_stats_trans ..'.format(ts, stats_file, freq))
        df = pd.read_csv(stats_file)
        if freq == 'hourly': # add day column
            df['day'] = df.apply(lambda row: row['time_label'].split()[0], axis=1)
        else: # for daily, just rename time_label to day
            df = df.rename(columns = {'time_label': 'day'})
        partition_col = 'day'

        df.drop(labels=partition_col, axis=1, inplace=True)
        spark_df = hc.createDataFrame(df)
        # convert NaN to null, as they are different in Hive
        cols = [when(~col(x).isin("NULL", "NA", "NaN"), col(x)).alias(x) for x in spark_df.columns]
        spark_df = spark_df.select(*cols)

        spark_df.registerTempTable('update_dataframe')
        day = pd.to_datetime(str(d)).date().strftime('%Y-%m-%d')  # convert date format
        sql_cmd = """SELECT COUNT(*) FROM guobiao_tsp_tbls.{}_stats_trans
                  WHERE {}='{}' """.format(freq, partition_col, day)
        print(sql_cmd)
        hc.sql(sql_cmd).show()

        sql_cmd = """INSERT INTO TABLE guobiao_tsp_tbls.{}_stats_trans
                  PARTITION ({}='{}')
                  SELECT * FROM update_dataframe""".format(freq, partition_col, day)
        print(sql_cmd)
        hc.sql(sql_cmd)

        sql_cmd = """SELECT COUNT(*) FROM guobiao_tsp_tbls.{}_stats_trans
                  WHERE {}='{}' """.format(freq, partition_col, day)
        print(sql_cmd)
        hc.sql(sql_cmd).show()

        ts = pd.to_datetime(datetime.now())
        print('{}: {} has been exported to Hive (overwrite mode)'.format(ts, stats_file))
        # write a marker file
        with open(out_flag_file, 'w') as f:
            pass
    sc.stop()
    print('done.')

In [1]:
__author__ = "Shuai Tang (stang@gacrndusa.com)"
__version__ = 0.3

import sys
sys.path.insert(0, 'stats-core/')
from utils import *
from vehicle import *

In [15]:
in_f = '20180101/by_vin/A5HEV_LMGHP1S5XH1000391.csv'
veh = Vehicle(in_f)

In [24]:
pd.options.display.max_columns = 100

In [30]:
veh.df.head()

,vin,vintype,ts,veh_st,veh_chargeSt,veh_runMode,veh_spd,veh_odo,veh_volt,veh_curr,veh_soc,veh_dcdcSt,veh_gear,veh_insulation,veh_pedal_deep,veh_pedalSt,dm_seq,dm_st,dm_ctl_temp,dm_spd,dm_torq,dm_temp,dm_ctl_volt,dm_ctl_dc_curr,eng_st,eng_spd,eng_consumption,loc_st,loc_lon84,loc_lat84,data_batt_subSys_volt_highest_seq,data_batt_sc_volt_highest_seq,data_batt_sc_volt_highest,data_batt_subSys_volt_lowest_seq,data_batt_sc_volt_lowest_seq,data_batt_sc_volt_lowest,data_batt_subSys_temp_highest_seq,data_batt_temp_probe_highest_seq,data_batt_temp_highest,data_batt_subSys_temp_lowest_seq,data_batt_temp_probe_lowest_seq,data_batt_temp_lowestest,alm_lvl_higest,alm_common_flag,alm_common_temp_diff,alm_common_temp_high,alm_common_esd_high,alm_common_esd_low,alm_common_soc_low,alm_common_sc_high,alm_common_sc_low,alm_common_soc_high,alm_common_soc_hop,alm_common_esd_unmatch,alm_common_sc_consistency,alm_common_insulation,alm_common_dcdc_temp,alm_common_brk,alm_common_dcdc_st,alm_common_dmc_temp,alm_common_HVIL_st,alm_common_dm_temp,alm_common_esd_charge_over,alm_esd_cnt,alm_esd_list,alm_dm_cnt,alm_dm_list,alm_eng_cnt,alm_eng_list,alm_others_cnt,alm_others_list,esd_volt_subsys_seq,esd_volt,esd_curr,esd_sc_cnt,esd_frame_start,esd_frame_sc_cnt,esd_frame_sc_list,esd_temp_probe_list
0,LMGHP1S5XH1000391,A5HEV,2018-01-01 00:00:05+08:00,2,1,1,0.0,5763.0,345.0,-19.5,58,1,1111,1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,110.040307,40.576519,1,15,4.842,1,21,4.830,1,24,12,1,33,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0,NaN,0,NaN,NaN,345.0,19.6,90,1,90,3.838 | 3.836 | 3.836 | 3.838 | 3.84 | 3.84 |...,7 | 8 | 8 | 9 | 9 | 9 | 9 | 11 | 11 | 11 | 11...
1,LMGHP1S5XH1000391,A5HEV,2018-01-01 00:00:15+08:00,2,1,1,0.0,5763.0,345.0,-17.5,58,1,1111,1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,110.040290,40.576496,1,14,4.842,1,21,4.831,1,24,12,1,33,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0,NaN,0,NaN,NaN,345.0,17.6,90,1,90,3.837 | 3.837 | 3.837 | 3.839 | 3.84 | 3.84 |...,7 | 8 | 8 | 9 | 9 | 9 | 10 | 10 | 11 | 11 | 1...
2,LMGHP1S5XH1000391,A5HEV,2018-01-01 00:00:25+08:00,2,1,1,0.0,5763.0,345.0,-17.0,58,1,1111,1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,110.040262,40.576466,1,14,4.841,1,21,4.831,1,24,12,1,33,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0,NaN,0,NaN,NaN,345.0,17.2,90,1,90,3.837 | 3.837 | 3.837 | 3.838 | 3.839 | 3.84 ...,7 | 8 | 8 | 9 | 9 | 9 | 9 | 10 | 11 | 11 | 11...
3,LMGHP1S5XH1000391,A5HEV,2018-01-01 00:00:45+08:00,2,1,1,0.0,5763.0,345.0,-19.5,58,1,1111,1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,110.040244,40.576427,1,6,4.842,1,21,4.832,1,24,12,1,33,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0,NaN,0,NaN,NaN,345.0,19.6,90,1,90,3.839 | 3.838 | 3.837 | 3.839 | 3.84 | 3.842 ...,7 | 8 | 8 | 9 | 9 | 9 | 9 | 10 | 11 | 11 | 10...
4,LMGHP1S5XH1000391,A5HEV,2018-01-01 00:00:55+08:00,2,1,1,0.0,5763.0,345.0,-17.5,58,1,1111,1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,110.040246,40.576425,1,7,4.842,1,21,4.831,1,24,12,1,33,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,0,NaN,0,NaN,0,NaN,NaN,345.3,18.1,90,1,90,3.838 | 3.838 | 3.837 | 3.839 | 3.84 | 3.84 |...,7 | 8 | 8 | 9 | 9 | 9 | 9 | 10 | 11 | 11 | 11...


In [1]:
import pandas as pd
import numpy as np
import findspark
findspark.init("/usr/hdp/current/spark2-client")
import pyspark

conf = pyspark.SparkConf().setAll([('spark.app.name', 'new_car_models'),
                                   ('spark.master', 'yarn-client'),
                                   ('spark.executor.memory', '10g'), 
                                   ('spark.executor.cores', '3'), 
                                   ('spark.executor.instances', '20'),
                                   ('spark.driver.memory', '10g')])

In [2]:
sc = pyspark.SparkContext.getOrCreate(conf=conf)
sc

<SparkContext master=yarn-client appName=new_car_models>

In [3]:
%%time
date = '20180101'
rdd = sc.textFile('hdfs://namenode:8020/data/guobiao/csv/d={}'.format(date))

CPU times: user 2.09 ms, sys: 1.36 ms, total: 3.45 ms
Wall time: 700 ms


In [ ]:
import pixiedust

In [ ]:
pixiedust.

In [3]:
pixiedust.installPackage("graphframes:graphframes:0")

AttributeError: 'module' object has no attribute 'installPackage'

In [2]:
pixiedust.installPackage("https://github.com/ibm-cds-labs/spark.samples/raw/master/dist/streaming-twitter-assembly-1.6.jar")

AttributeError: 'module' object has no attribute 'installPackage'

In [8]:
%%scala
rdd.zipWithIndex().map({case (x,y) => (y,x)}).sortByKey(ascending = false).map({case (x,y) => y}).take(10).reverse

UsageError: Cell magic `%%scala` not found.


In [3]:
res = rdd.map(lambda line: (line.split(',')[1], line)).\
    groupByKey().mapValues(list).collect()

for model, val in res:
    # each val is a list of records for that model
    with open('{}_{}_part0.csv'.format(model, date), 'w') as f:
        for record in val:
            f.write(record + '\n')

CPU times: user 7.7 s, sys: 5.38 s, total: 13.1 s
Wall time: 1min 2s


In [4]:
df = pd.read_csv('A2APHEV_{}.csv'.format(date), header=None)
df[2] = pd.to_datetime(df[2], unit='ms', utc=True).dt.tz_convert('Asia/Hong_Kong')
df = df.sort_values(by=df.columns[2])

df['len(voltage)'] = df[77].apply(lambda x: len(x.split('|')))
df['len(temp)'] = df[78].apply(lambda x: len(x.split('|')))

In [5]:
df.head(1)

,0,1,2,3,4,5,6,7,8,9,...,71,72,73,74,75,76,77,78,len(voltage),len(temp)
254816,LMGAJ1S28H1000897,A2APHEV,2017-11-06 05:21:05+08:00,3,3,1,0.0,16.0,316.0,0.0,...,NaN,316.0,1.4,88,1,88,3.585 | 3.585 | 3.586 | 3.585 | 3.586 | 3.585...,24 | 24 | 24 | 24 | 24 | 24 | 24 | 24 | 25 | ...,88,15


In [6]:
print(df.shape)
tmp = df[df.iloc[:, 2] < pd.to_datetime(date).tz_localize('Asia/Hong_Kong')].iloc[:, 2]
tmp = tmp.apply(lambda x: x.date())
tmp.value_counts()

(367941, 81)


2018-02-07    112
2018-02-03    108
2018-01-05    102
2018-02-06     85
2017-12-22     57
2018-01-09     51
2018-01-29     30
2018-01-10     27
2018-01-27      3
2018-01-25      2
2018-01-13      2
2018-01-07      2
2017-11-06      1
2017-12-29      1
2018-01-12      1
2018-01-11      1
Name: 2, dtype: int64

In [7]:
print(df['len(voltage)'].value_counts())
print('------------------------------------')
print(df['len(temp)'].value_counts())

88    322273
96     45668
Name: len(voltage), dtype: int64
------------------------------------
16    243996
12     75909
8      44921
15      1736
1        624
11       613
7         84
5         27
4         11
2          9
3          8
6          3
Name: len(temp), dtype: int64


In [8]:
df[df['len(temp)'] == 8].iloc[:5, [0,1,2,80]]

,0,1,2,len(temp)
176376,LMGAJ1S86J1002237,A2APHEV,2018-02-08 00:00:05+08:00,8
210167,LMGAJ1S88H1002024,A2APHEV,2018-02-08 00:00:05+08:00,8
350519,LMGAJ1S82H1001953,A2APHEV,2018-02-08 00:00:05+08:00,8
172289,LMGAJ1S86H1001924,A2APHEV,2018-02-08 00:00:05+08:00,8
337359,LMGAJ1S81H1002012,A2APHEV,2018-02-08 00:00:05+08:00,8


In [9]:
df[df['len(voltage)'] == 96].iloc[:5, [0,1,2,79]]

,0,1,2,len(voltage)
176376,LMGAJ1S86J1002237,A2APHEV,2018-02-08 00:00:05+08:00,96
210167,LMGAJ1S88H1002024,A2APHEV,2018-02-08 00:00:05+08:00,96
350519,LMGAJ1S82H1001953,A2APHEV,2018-02-08 00:00:05+08:00,96
172289,LMGAJ1S86H1001924,A2APHEV,2018-02-08 00:00:05+08:00,96
337359,LMGAJ1S81H1002012,A2APHEV,2018-02-08 00:00:05+08:00,96
